Objetos, funciones y constantes útiles para la gestión del AVBD

In [1]:
#Constantes utiles
PATH="Path-to-root-of-AVBD-directories"
ANNO='Directory-of-actual-AVBD'

#Niveles de objetivos y competencias definidos en las encuestas Mooodle
niveles_obxectivos=[
    'Desenvolveu satisfactoriamente todos os obxectivos',
    'Desenvolveu suficientemente todos os obxectivos',
    'Non acada todos pero apréciase madureza e posibilidades de acadar o grao esixido',
    'Non acada e non se aprecia madureza e posibilidades de acadar o grao esixido',
    'Non é alumno'
]

niveles_competencias=[
    'Alto',
    'Medio',
    'Baixo',
    'NON conseguido',
    'Non é alumno'
    
]

In [5]:
#Definicion de la clase BD


import sqlite3

class BD(object):
    def __init__(self,nome):
        self.conn=sqlite3.connect(nome)
        self.cur=self.conn.cursor()
        
    def __del__(self):
        
        self.conn.close()
        
    def ejecuta(self,sent):
        self.cur.execute(sent)
        
    def commit(self):
        self.conn.commit()
        
    def busca(self,sent):
        self.ejecuta(sent)
        return self.cur.fetchall()

class AV_BD(BD):
    '''Objeto para la gestión de AVBD'''
    
    def __init__(self,nome):
        
        #diccionario con las definiciones SQL de cada tabla de la BD
        
        self.TABLES={'Alumno_IP':'''CREATE TABLE "Alumno_IP" (
            RolId	INTEGER UNIQUE,
            direccion	TEXT,
            rep_legal	TEXT,
            Telf_contacto	TEXT,
            data_nacemento	TEXT,
            AVlogin	TEXT UNIQUE,
            AVId	TEXT UNIQUE,
            AVmail	TEXT
        )''',
       'Asignatura':'''CREATE TABLE "Asignatura" (
id	INTEGER PRIMARY KEY AUTOINCREMENT,
DptoId	INTEGER,
nome	TEXT UNIQUE,
abrev	TEXT UNIQUE
)''',
       'Aulas':'''CREATE TABLE "Aulas" (
id	INTEGER PRIMARY KEY AUTOINCREMENT,
nome	TEXT NOT NULL UNIQUE
)''',
       'Competencias':'''CREATE TABLE "Competencias" (
RolId	INTEGER NOT NULL,
DocenteId	INTEGER,
ListaCompId	INTEGER NOT NULL,
data	TEXT NOT NULL,
aval	TEXT NOT NULL,
valor	INTEGER NOT NULL,
UNIQUE (RolId,ListaCompId,aval,DocenteId)
)''',
       'Docente_IP':'''CREATE TABLE "Docente_IP" (
RolId	INTEGER UNIQUE,
DptoId	INTEGER,
substitue	INTEGER,
email	TEXT UNIQUE,
AVmail	TEXT,
AVlogin	TEXT UNIQUE,
AVId	TEXT UNIQUE
)''',
       'Dpto':'''CREATE TABLE "Dpto" (
id	INTEGER PRIMARY KEY AUTOINCREMENT,
nome	TEXT NOT NULL UNIQUE
)''',
       'ListaComp':'''CREATE TABLE ListaComp (
id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
nome	TEXT UNIQUE,
abrev	TEXT UNIQUE
)''',
       'Listados':'''CREATE TABLE Listados (
        RolId INTEGER  NOT NULL,
        AulasId INTEGER  NOT NULL,
        AsigId INTEGER  NOT NULL,
        rol TEXT  NOT NULL,
        UNIQUE (RolId,AulasId,AsigId)
        )''',
       'Resultados':'''CREATE TABLE "Resultados" (
RolId	INTEGER NOT NULL,
AsigId	INTEGER NOT NULL,
data	TEXT,
aval	TEXT NOT NULL,
nota	TEXT NOT NULL,
UNIQUE (RolId,AsigId,aval)
)''',
       'Rol':'''CREATE TABLE "Rol" (
id	INTEGER PRIMARY KEY AUTOINCREMENT,
nome	TEXT  NOT NULL,
apelidos	TEXT  NOT NULL,
status	INTEGER,
UNIQUE(nome,apelidos)
)''',
   'Forum':'''CREATE TABLE `Forum` (
`rolId`	INTEGER NOT NULL,
`docId`	INTEGER NOT NULL,
`data`	TEXT NOT NULL,
`coment`	TEXT,
UNIQUE (rolId,docId,data)
);
   '''}
        
        self.nome=nome
        try:
            
            self.conn = sqlite3.connect(nome)
            self.cur = self.conn.cursor() 
            
            
        except:
                  
            print('No se puede conectar '+nome)
   
        
        
        
    
    def resetBD(self,llimp=[]):
        '''
        Crea una nueva BD o resetea alguna tabla de una BD existente
        nome: path+nombre archivo SQLite
        llimp: lista de tablas a resetear. Si la lista está vacia se crea una nueva base de datos 
        '''
        
        
        
        #diccionario con las definiciones SQL de cada tabla de la BD
        tables=self.TABLES
        

        #Si llimp esta vacio crear una nueva BD
        if not llimp:
            llimp=list(tables.keys())

        #Si algún nombre no es una tabla de la BD abortar
        for table in llimp:
            if table not in tables.keys():

                return table+' non é unha tabla de AV_BD'


        

        #Crear la BD
        for tabla in llimp:
            self.cur.execute('DROP TABLE IF EXISTS %s' % tabla)
            self.cur.executescript(tables[tabla])
            #Si en el directorio raiz(PATH) hay un csv con el nombre de una tabla
            #se cargan los valores del archivo en la BD
            try:
                fich=open(self.path+tabla+'.csv','r')

            except:
                continue
            
            #Extrae los nombres de los campos de la definicion de la tabla
            campos=[item.strip('\t').split()[0] for item in tables[tabla].strip().split('\n')[2:-1] if 'UNIQUE' not in item.strip().split()[0]]
            
            for item in fich:
                cad='INSERT INTO %s (%s) VALUES ("%s")' % (tabla,', '.join(campos), '", "'.join(item.strip().split(';')))
                self.cur.execute(cad)
            fich.close()
            
        #Si es posible introducir la asignatura "Tutoría" del Dpto "Orientación"
        if 'Asignatura' in llimp:
            cad='SELECT id FROM Asignatura WHERE nome="Tutoría"'
            self.cur.execute(cad)
            AsigId=self.cur.fetchone()
            cad='SELECT id FROM Dpto WHERE nome="Orientación"'
            self.cur.execute(cad)
            DptoId=self.cur.fetchone()
            if DptoId and not AsigId:
                cad='INSERT INTO Asignatura(DptoId,nome,abrev) VALUES (%i,"%s","%s")' % (DptoId[0],'Tutoría','tut.')
                self.cur.execute(cad)

        self.conn.commit()
           

    def buscaRolId(self,nome='',apelido='',append=False):
        '''
        buscaRolId(nome=str[''], apelido=str[''], append=False)
        Busca el rol Id de todos los usuarios que coincidan con 
        las cadenas nome y/o apelido suministradas.
        En la busqueda se usan los comodines % y la función LIKE
        Si append es True y el usuario no existe en la BD se añade el usuario nome, apelido al Rol'''
        
        

        cad='SELECT id FROM Rol WHERE '
        if nome:
            cad+='nome LIKE ("%s") ' % nome.join(['%','%'])
        if apelido:
            if nome:
                cad+='AND '
            cad+='apelidos LIKE ("%s")' % apelido.join(['%','%'])
        #print(cad)
        self.cur.execute(cad)
        aux=self.cur.fetchall()
        if aux:
            RolId=aux
        elif append and nome and apelido:

            cad='INSERT INTO Rol(nome,apelidos,status) VALUES ("%s","%s",1)' % (nome,apelido)
            #print(cad)
            self.cur.execute(cad)

            RolId=self.buscaRolId(nome,apelido)
        else:
            return None


        return RolId
    
    def extraeGrupo(self, grupo=None ,asig='tut.' ,rol='alumno'):
        '''
        extraeGrupo(grupo=None,asig='tut.',rol='alumno')
        Extrae el listado del grupo/asignatura con un determinado rol
        grupo: cadena,debe ser uno de los nombres definidos en la tabla Aulas
        asig: cadena de la abreviatura de la asignatura como se definen en la tabla Asignatura
        rol: cadena, uno de: alumno/docente/tutor
        '''
        
        if not (grupo or asig):
            return None
        cad="SELECT  Rol.id, Rol.nome, Rol.apelidos  FROM   Rol join Listados join Aulas "
        if asig:
            cad+="join Asignatura "
        cad+="on Rol.id=Listados.RolId and Aulas.id=Listados.AulasId "
        if asig:
            cad+="and Listados.AsigId=Asignatura.id "
        cad+="WHERE "
        if grupo:
            cad+='Aulas.nome="%s" ' % grupo
        if asig:
            if grupo:
                cad+="AND "
            cad+='Asignatura.abrev="%s" ' % asig
        cad+='AND rol="%s"' % rol
        return self.busca(cad)
    
    def cambiaGrupo(self,listado):
        '''
        cambiaGrupo(listado)
        listado [(nome1,apelido1,ngrupo1),...]
        Cambia al grupo ngrupo1 al usuario nome1,apelido1
        '''
        for nome,apelido,ngrupo in listado:
            RolId=self.buscaRolId(nome,apelido)
            if not RolId:
                print('El alumno %s es desconocido' % ' '.join((nome,apelido)))
                continue
            RolId=RolId[0][0]
            self.cur.execute('SELECT id FROM Aulas WHERE nome="%s"' % ngrupo)
            AulasId=self.cur.fetchone()
            if not AulasId:
                print('El grupo %s es desconocido' % ngrupo)
                continue
            AulasId=AulasId[0]
            self.cur.execute('UPDATE Listados SET AulasId=%i WHERE RolId=%i' % (AulasId,RolId))
        self.conn.commit()
        
    def alta(self,nuevos):
        '''
        Da de alta a una lista de nuevos alumnos en el AVBD 
        Devuelve la lista de nuevos alumnos para dar de alta en el AV
        nuevos: lista de tuplas (nome,apelidos,grupo)
        '''

        nnew=[]
        for nome,apelido,grupo in nuevos:
            RolId=self.buscaRolId(nome,apelido)
            if RolId:
                print ('%s %s ya está en BD' % (nome, apelido))
                AVlogin=self.busca('SELECT AVlogin FROM Alumno_IP WHERE RolId=%i' % RolId[0][0])
                if AVlogin:
                    print('%s %s ya esta dado de alta en AV como %s' % (nome,apelido,AVlogin[0][0]))
                else:
                    nnew.append[(nome,apelido)]
            else:
                RolId=self.buscaRolId(nome,apelido,append=True)
                self.ejecuta('INSERT INTO Alumno_IP(Rolid) VALUES (%i)' % RolId[0][0])
                AulasId=self.busca('SELECT id FROM Aulas WHERE "%s" LIKE nome' % grupo)[0][0]
                self.ejecuta('INSERT INTO Listados VALUES (%i,%i,1,"alumno")' % (RolId,AulasId))
                
                nnew.append[(nome,apelido)]
        self.commit()
        return nnew
    
    def baja(self,bajas,tipo='alum'):
        '''
        Da de baja a una lista de alumnos(docentes) en el AVBD 
        y genera el fich de bajas para el AV
        
        bajas: lista de tuplas (nome,apelidos)
        tipo: alum/doc
        '''
        bajAV=[]
        for nome, apelido in bajas:
            RolId=self.buscaRolId(nome,apelido)
            if not RolId:
                print('Alumno desconocido {1},{0}'.format(nome,apelido))
            else:
                RolId=RolId[0][0]
            self.cur.execute("SELECT AVlogin FROM Alumno_IP WHERE ROLID=%i" % RolId)
            bajAV.append((self.cur.fetchone()[0],nome,apelido)  )
            for tabla in [ 'Alumno_IP', 'Listados', 'Competencias', 'Resultados']:
                self.cur.execute("DELETE FROM %s WHERE RolId=%i" % (tabla,RolId))

            self.cur.execute("DELETE FROM Rol WHERE id=%i" % RolId)
        self.commit()
        return bajAV
    
   
    def colapsar(self,colapsar):
        '''
        Colapsar duplicaciones debidas a ortografia de nombre/apellidos
        colapsar: lista de tuplas con los RolId a colapsar [(RolId viejo,RolId nuevo),....]
        Se combina la información de los dos registros y se elimina el usuario con RolId viejo
        '''
        for tupla in colapsar:
            #Colapsa la IP
            viejo=self.busca('SELECT * FROM Alumno_IP WHERE RolId=%i' % tupla[0])[0][1:]
            nuevo=self.busca('SELECT * FROM Alumno_IP WHERE RolId=%i' % tupla[1])[0][1:]
            
            mix=[]
            for v,n in zip(viejo,nuevo):
                if n:
                    mix.append(n)
                elif v:
                    mix.append(v)
                else:
                    mix.append('')
            
            #Antes hay que borrar los dos registros para evitar UNIQUE
            self.ejecuta('DELETE FROM Alumno_IP WHERE RolId=%i' % tupla[0])
            self.ejecuta('DELETE FROM Alumno_IP WHERE RolId=%i' % tupla[1])
            
            #insertar el mix
            cad='INSERT INTO Alumno_IP(RolId,direccion, rep_legal, Telf_contacto, data_nacemento, AVlogin, AVId, AVmail) VALUES (%i,"%s","%s", "%s", "%s","%s", "%s", "%s")' % tuple([tupla[1]]+mix)
            self.ejecuta(cad)
            
            #Eliminar el rol viejo
            self.ejecuta('DELETE FROM Rol WHERE id=%i' % tupla[0])

            #Colapsa otros posibles registros del alumno
            for tabla in ['Competencias','Listados','Resultados']:

                #Extrae los nombres de los campos de la definicion de la tabla
                campos=[item.strip('\t').split()[0] for item in self.TABLES[tabla].strip().split('\n')[2:-1] if 'UNIQUE' not in item.strip().split()[0]]
                
                viejos=self.busca('SELECT %s FROM %s Where RolId=%i' % (', '.join(campos[1:-1]),tabla,tupla[0]))
                nuevos=self.busca('SELECT %s FROM %s Where RolId=%i' % (', '.join(campos[1:-1]),tabla,tupla[1]))
                
                #Actualizar los registros viejos 
                if viejos:
                    viejos=set(viejos)
                    if nuevos:
                        nuevos=set(nuevos)
                        viejos=(viejos-nuevos)
                    if viejos:
                        for item in list(viejos):
                            cad='UPDATE %s SET RolId=%i WHERE ' % (tabla,tupla[1])
                            cad+='RolId=%i ' % tupla[0]
                            for text,value in zip(campos[1:-1],item):
                                cad+='%s="%s" ' % (text,value)
                            self.ejecuta(cad)
                    
        self.conn.commit()
        
    


In [3]:
#Funciones utiles
def sinAcento(text):
    '''
    Elimina las vocales acentuadas y la ñ de un texto
    '''
    import re
    sust=[('Á','A'),('á','a'),('É','E'),('é','e'),('Í','I'),('í','i'),('Ó','O'),('ó','o'),('ú','u'),('Ú','U'),('ñ','nn'),('Ñ','NN')]
    for pat,res in sust:
        text=re.sub(pat,res,text)
    return text


def ordenStandar(lista):
    '''
    Ordena una lista de tuplas (salida extraeGRupo) por orden alfabético de apellidos y nombres ignorando acentos
    '''
    origen={item[0]:(item[1],item[2]) for item in lista}
    orden={(sinAcento(item[1].lower()),sinAcento(item[2].lower())):item[0] for item in lista}
    keys=list(orden.keys())
    keys.sort(key=lambda x: x[0])
    keys.sort(key=lambda x: x[1])
    
    return [(orden[key],origen[orden[key]][0],origen[orden[key]][1]) for key in keys]
    
def nomtohtml(n):
    '''
    converts utf names to html names
    '''
    from re import sub
    
    acute=[
        ['á','a'],['Á','A'],['é','e'],['É','E'],['í','i'],['Í','I'],['ó','o'],['Ó','O'],['ú','u'],['Ú','U']
    ]
    tilde=[
        ['ñ','n'],['Ñ','N']
    ]
    abrev=[
        ['ª','&ordf;'],
        ['º', '&deg;']
    ]
    dieresis=[
        ['ü','u'],['Ü','U']
    ]
    #print n
    for item in abrev:
        n=sub(item[0],item[1],n)
    for item in acute:
        n=sub(item[0],'&'+item[1]+'acute;',n)
    #print n
    for item in tilde:
        n=sub(item[0],'&'+item[1]+'tilde;',n)
    for item in dieresis:
        n=sub(item[0],'&'+item[1]+'uml;',n)
    
    #print n
    return n     
        
def extrae_data(nome,sep=';',okhead=True):
    '''
    Extrae datos de un csv
    '''
    fich=open(nome,'r')
    rawdata=fich.readlines()
    if okhead:
        head=[value.strip('" \n') for value in rawdata.pop(0).strip('\n "').split(sep)]
    else:
        head=None
    data=[]
    for text in rawdata:
        data.append([value.strip('" \n') for value in text.strip('"\n ').split(sep)])
    return data, head



def genera_AVlogin(listado,login=''):
    '''
    Genera el login para el AV. Devuelve una lista de tuplas [(nome,apelido,AVlogin),...]
    login: lista con los login actuales en uso
    listado: [(nome,apelido),...] donde nome, apelido usuario alta
    listaAV: lista de tuplas [(nome,apelido,AVlogin),...] que se usa como entrada de altas_AV
    '''
    import re
    
    listaAV=[]
    for nome,apelido in listado:
        stop=1

        nome=sinAcento(nome.lower())
        apelido=sinAcento(apelido.lower())

        lnome=nome.split()
        lapelido=apelido.split()
        AVlogin=lnome[0]
        for item in lnome[1:]:
            AVlogin+=item[0:stop]
        AVlogin+='.'
        for item in lapelido:
            AVlogin+=item[0:stop]
        while AVlogin in login:
            stop+=1
            AVlogin=lnome[0]
            for item in lnome[1:]:
                AVlogin+=item[0:stop]
            AVlogin+='.'
            for item in lapelido:
                AVlogin+=item[0:stop]
            
        listaAV.append((nome,apelido,AVlogin))
        login.append(AVlogin)
    return listaAV

def altas_AV(nomfich,nuevos,logins=''):
    '''
    Genera archivo para la matriculacion masiva en Moodle
    nomfich: PATH+nombre fich salida, archivo csv separado por ";"
    nuevos: listas de tuplas (nome,apelido)
    logins: logins en uso en la AV
    '''
    fich=open(nomfich,'w')
    fich.write('username;password;firstname;lastname;email;city\n')
    for item in genera_AVlogin(nuevos,logins):
        nome,apelido,AVlogin=item
        
        fich.write(';'.join([AVlogin,'1234',nome,apelido,'nomail@nowhere.com','Alumno\n']))

    fich.close()
    
def obxectivos(PATH,curso,alumns,niveles=niveles_obxectivos,required=1):
    '''
    crea la encuesta para consecución de objetivos
    fichero xml con la plantilla para las encuestas de consecución de objetivos 
    PATH: ruta a los ficheros de salida
    curso: grupo
    alumns: lista de tuplas (apelidos, nome)
    niveles: niveles considerados
    required: 1 si es obligatorio responder a todos
    '''
    cad_items='\n|'.join(niveles)
    fich=open(PATH+curso+'_obxectivos.xml','w')
    fich.write('<?xml version="1.0" encoding="UTF-8" ?>\n')
    fich.write('<FEEDBACK VERSION="200701" COMMENT="XML-Importfile for mod/feedback">\n\t<ITEMS>\n')
    for alumn in alumns:
        fich.write('\t\t<ITEM TYPE="multichoice" REQUIRED="%i">\n' % required)
        fich.write('\t\t\t<ITEMTEXT>\n\t\t\t\t<![CDATA[%s]]>\n\t\t\t </ITEMTEXT>\n' % ', '.join(alumn))
        fich.write('\t\t\t<PRESENTATION>\n')
        fich.write('\t\t\t\t<![CDATA[r>>>>>%s]]>\n' % cad_items)
        fich.write('\t\t\t</PRESENTATION>\n\t\t</ITEM>\n')
    fich.write('\t</ITEMS>\n</FEEDBACK>\n')
    fich.close()


def competencias(PATH,curso,alumns,niveles=niveles_competencias,required=1):
    '''
    crea la encuesta para competencias
    fichero xml con la plantilla para las encuestas de competencias
    PATH: ruta a los ficheros de salida
    curso: grupo
    alumns: lista de tuplas (apelidos, nome)
    niveles: niveles considerados
    required: 1 si es obligatorio responder a todos
    '''
    cad_items='\n|'.join(niveles)
    fich=open(PATH+curso+'_competencias.xml','w')
    fich.write('<?xml version="1.0" encoding="UTF-8" ?>\n')
    fich.write('<FEEDBACK VERSION="200701" COMMENT="XML-Importfile for mod/feedback">\n\t<ITEMS>\n')
    for alumn in alumns:
        fich.write('\t\t<ITEM TYPE="multichoice" REQUIRED="%i">\n' % required)
        fich.write('\t\t\t<ITEMTEXT>\n\t\t\t\t<![CDATA[%s]]>\n\t\t\t </ITEMTEXT>\n' % ', '.join(alumn))
        fich.write('\t\t\t<PRESENTATION>\n')
        fich.write('\t\t\t\t<![CDATA[r>>>>>%s]]>\n' % cad_items)
        fich.write('\t\t\t</PRESENTATION>\n\t\t</ITEM>\n')
    fich.write('\t</ITEMS>\n</FEEDBACK>\n')
    fich.close()
    
def listadoAVlogin(nomfich,listado):
    '''
    Genera la tabla HTML  alumno-->AVlogin
    nomfich: nombre del archivo de salida (con path)
    listado: lista de tuplas [(apelidos,nome,avlogin),...]'''
    fich=open(nomfich,'w')
    fich.write('''<html>
    <body>
    <table border=1>
    <tr>
    <th>Nome</th>
    <th>login</th>
    </tr>
    ''')
    for item in listado:
        
        fich.write('<tr>\n\t<td>%s</td><td>%s</td>\n</tr>\n' % (nomtohtml(', '.join(item[:2])),item[-1]))
    fich.write('</table>\n</body>\n</html>')
    fich.close()